# Bokeh Examples

In this notebook, we're just going to attempt to create a few bokeh plots using some dummy data.

In [1]:
import datetime
import random
from pandas import DataFrame, melt

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LinearColorMapper, HoverTool
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral5, Colorblind5, Viridis6, Magma6
from bokeh.models.tools import HoverTool

output_notebook()

Loading BokehJS ...

## Generate some datetime data and some network data

In [2]:
num_points = 1000
current_datetime = datetime.datetime.strptime('01/01/2019 00:00:00', '%m/%d/%Y %H:%M:%S')
categories = ['item', 'msg', 'sound', 'email', 'alert']
nodes = ['aa', 'bb', 'cc', 'dd', 'ef', 'gi',' hs', 'jk', 'lol', 'ms']


datetimes = []
datetimes_alpha = []
edges = []
# generate the datetimes first
for item in range(num_points):
    # add the current time to our collection and calculate the next one
    dt_entry = {'datetime': current_datetime}
    dt_alpha_entry = {'datetime': current_datetime}
    current_datetime += datetime.timedelta(days=1)
    
    # generate the category flag
    for category in categories:
        dt_entry[category] = random.randrange(2)
        dt_alpha_entry[category] = random.random()
        
    datetimes.append(dt_entry)
    datetimes_alpha.append(dt_alpha_entry)
    
# now generate the nodes
max_weight = 30
for node in nodes:
    for edge in random.sample(range(num_points), random.randrange(num_points)):
        source_type, destination_type = random.choices(categories, k=2)
        edges.append({'source': node, 'destination': edge, \
                      'weight': random.randrange(1, max_weight), \
                      'source_type': source_type, 'destination_type': destination_type})
        

datetimes_df = DataFrame(datetimes)
edges_df = DataFrame(edges)

In [3]:
datetimes_df.head()

,datetime,item,msg,sound,email,alert
0,2019-01-01,1,0,0,1,1
1,2019-01-02,0,0,0,1,1
2,2019-01-03,1,1,1,0,0
3,2019-01-04,1,1,1,0,1
4,2019-01-05,0,0,1,1,0


In [4]:
reshaped_dt_df = melt(datetimes_df, id_vars=['datetime'], value_vars=categories)
reshaped_dt_df = reshaped_dt_df[reshaped_dt_df['value'] == 1]
reshaped_dt_df.head()

,datetime,variable,value
0,2019-01-01,item,1
2,2019-01-03,item,1
3,2019-01-04,item,1
5,2019-01-06,item,1
11,2019-01-12,item,1


In [5]:
reshaped_dt_alpha_df = melt(DataFrame(datetimes_alpha), id_vars=['datetime'], value_vars=categories)
reshaped_dt_alpha_df = reshaped_dt_alpha_df[reshaped_dt_alpha_df['value'] > 0]
reshaped_dt_alpha_df.head()

,datetime,variable,value
0,2019-01-01,item,0.569788
1,2019-01-02,item,0.422299
2,2019-01-03,item,0.094694
3,2019-01-04,item,0.899054
4,2019-01-05,item,0.799138


In [6]:
edges_df.head()

,source,destination,weight,source_type,destination_type
0,aa,342,10,alert,sound
1,aa,159,9,sound,msg
2,aa,333,15,alert,msg
3,aa,120,28,msg,email
4,aa,20,12,sound,alert


## Datetime plots

In [7]:
# a plot using symbols to represent an occurence of an event type on each day
source = ColumnDataSource(reshaped_dt_df)

p = figure(plot_width=700, plot_height=150, y_range=categories, x_axis_type='datetime', 
           title="Days on which we see an item of each type", toolbar_location='above')
cyl_cmap = factor_cmap('variable', palette=Colorblind5, factors=categories)

p.dash(x='datetime', y='variable', source=source, angle=90, angle_units='deg', \
       line_color=cyl_cmap, size=10)

p.xaxis[0].formatter.days = ['%M/%Y']
p.x_range.range_padding = 0
p.ygrid.grid_line_color = None
p.xgrid.grid_line_color = None

show(p)

In [8]:
# a plot using symbols to represent an occurence of an event type on each day and coloring the
# symbol based on a value
source = ColumnDataSource(reshaped_dt_alpha_df)

p = figure(plot_width=700, plot_height=150, y_range=categories, x_axis_type='datetime', 
           title="Days on which we see an item of each type (colored by value)", \
           toolbar_location='above')

color_mapper = LinearColorMapper(palette=list(reversed(Magma6)), low=0, high=1)
p.dash(x='datetime', y='variable', source=source, angle=90, angle_units='deg', \
       size=10, color={'field': 'value', 'transform': color_mapper})

p.xaxis[0].formatter.days = ['%M/%Y']
p.x_range.range_padding = 0
p.ygrid.grid_line_color = None
p.xgrid.grid_line_color = None

show(p)

In [9]:
# a vertical bars plot
source = ColumnDataSource(reshaped_dt_alpha_df[(reshaped_dt_alpha_df['variable'] == 'alert') \
                                               & (reshaped_dt_alpha_df['datetime'] < datetime.datetime.strptime('06/01/2019 00:00:00', '%m/%d/%Y %H:%M:%S'))])

p = figure(plot_width=700, plot_height=150, x_axis_type='datetime', 
           title="Days on which we have alerts", \
           toolbar_location='above')

p.vbar(x='datetime', top='value', source=source)

p.xaxis[0].formatter.days = ['%M/%Y']
p.x_range.range_padding = 0
p.ygrid.grid_line_color = None
p.xgrid.grid_line_color = None

show(p)